In [ ]:
%pylab notebook

## Solution to exercise 5: Simple Hay et al. model neuron simulation with one synaptic input

Move downwards and follow instructions to see solutions to various tasks:

In [ ]:
import os
retval = os.getcwd()
print("Current working directory %s" % retval)
os.chdir("hay_model/mod/")
!nrnivmodl
os.chdir(retval)

from os.path import join
import numpy as np
import neuron
import LFPy
from hay_model.hay_active_declarations import active_declarations
nrn = neuron.h

In [ ]:
def return_cell(synaptic_y_pos=900, conductance_type='active', weight=0.001, input_spike_train=np.array([10.])):
    """
    Runs a NEURON simulation and returns an LFPy cell object for a single synaptic input.
    :param synaptic_y_pos: position along the apical dendrite where the synapse is inserted.
    :param conductance_type: Either 'active' or 'passive'. If 'active' all original ion-channels are included,
           if 'passive' they are all removed, yielding a passive cell model.
    :param weight: Strength of synaptic input.
    :param input_spike_train: Numpy array containing synaptic spike times
    :return: cell object where cell.imem gives transmembrane currents, cell.vmem gives membrane potentials.
             See LFPy documentation for more details and examples.
    """
    nrn('forall delete_section()')
    model_path = join('hay_model')
    neuron.load_mechanisms(join(model_path, 'mod'))
    cell_parameters = {
        'morphology': join(model_path, 'cell1.hoc'),
        'v_init': -65,
        'passive': False,
        'nsegs_method': 'lambda_f',
        'lambda_f': 100,
        'dt': 2**-3,  # Should be a power of 2
        'tstart': -200,
        'tstop': 200,
        'custom_code': [join(model_path, 'custom_codes.hoc')],
        'custom_fun': [active_declarations],
        'custom_fun_args': [{'conductance_type': conductance_type}],
    }
    cell = LFPy.Cell(**cell_parameters)
    synapse_parameters = {
        'idx': cell.get_closest_idx(x=0., y=synaptic_y_pos, z=0.),  # Returns compartment on cell closest to coordinates
        'e': 0.,
        'syntype': 'ExpSyn',
        'tau': 10.,
        'weight': weight,
        'record_current': True,
    }
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(input_spike_train)
    cell.simulate(rec_imem=True, rec_vmem=True)
    return cell

In [ ]:
def plot_cell(cell):
    cell_plot_idx = 0
    subplot(121, aspect='equal')
    [plot([cell.xstart[idx], cell.xend[idx]], [cell.ystart[idx], cell.yend[idx]], c='k') 
     for idx in range(cell.totnsegs)]
    plot(cell.xmid[cell.synidx], cell.ymid[cell.synidx], 'ro')
    subplot(222, title='Membrane potential')
    plot(cell.tvec, cell.vmem[cell_plot_idx, :])
    subplot(224, title='Transmembrane currents')
    plot(cell.tvec, cell.imem[cell_plot_idx, :])
    savefig('exercise_5_1.png')

In [ ]:
def return_soma_v_line(cell, axes, dist):
    cell_plot_idx = 0
    sig_shifted = (cell.vmem[cell_plot_idx, :] - cell.vmem[cell_plot_idx, 0])
    axes[0].plot(cell.tvec, sig_shifted/np.max(sig_shifted), label='%d $\mu m$' % dist)
    axes[1].plot(cell.tvec, cell.vmem[cell_plot_idx, :], label='%d $\mu m$' % dist)

In [ ]:
def plot_v_response_with_distance(conductance_type):
    distances = np.linspace(0, 1200, 5)
    close('all')
    fig = figure()
    fig.text(0.5, 0.95, conductance_type.capitalize() + ' conductances', ha='center', size=17)
    ax1 = subplot(121, ylim=[-0.2, 1.2])
    ax2 = subplot(122, ylim=[-65.2, -64])
    for dist in distances:
        cell = return_cell(dist, conductance_type)
        return_soma_v_line(cell, [ax1, ax2], dist)
    legend()
    savefig('exercise_5_2_%s.png' % conductance_type)

In [ ]:
def coincidence_detection(conductance_type):
    #input_spike_train = np.array([20, 50, 80])
    input_spike_train = np.array([20, 60, 100])

    cell = return_cell(conductance_type=conductance_type, 
                       input_spike_train=input_spike_train, 
                       synaptic_y_pos=1100)
    input_idx = cell.synidx[0]
    soma_idx = 0
    close('all')
    fig = figure()

    fig.text(0.5, 0.95, conductance_type.capitalize() + ' conductances', ha='center', size=17)
    subplots_adjust(wspace=0.5, hspace=0.5)
    subplot(121, aspect='equal', frameon=False, xticks=[])
    [plot([cell.xstart[idx], cell.xend[idx]], [cell.ystart[idx], cell.yend[idx]], c='k') 
     for idx in range(cell.totnsegs)]
    plot(cell.xmid[cell.synidx], cell.ymid[cell.synidx], 'ro')

    subplot(222, title='Input\nmembrane potential', ylim=[-70, -35])
    plot(cell.tvec, cell.vmem[input_idx, :])

    subplot(224, title='Soma\nmembrane potential', ylim=[-65.02, -64.6])
    plot(cell.tvec, cell.vmem[soma_idx, :])
    savefig('exercise_5_3_%s.png' % conductance_type)
    print(input_idx)

__(i)__ For the simplest example of a single spike, uncomment the two first lines in the cell below and run.

__(ii)__ To see the somatic response to increasingly distant apical synaptic input in a passive neuron, uncomment line 1 and 3 in the cell below and run (making several plots at the same time gives figure conflicts, but you may save figures for comparison). To see the same for an active neuron, uncomment lines 1 and 4 and run. We note that the width of the synaptic response increases with distance for the passive case, but not (or much less) for the active case. For the active model, you can also see that the somatic voltage trace resulting from a distal apical input contains a hyperpolarized overshoot (or 'undershoot') after the initial depolarization. This is caused by the IH conductance, and you can read more about it in the reference: Williams SR & Stuart GJ (2000): Site Independence of EPSP Time Course Is Mediated by Dendritic Ih in Neocortical Pyramidal Neurons. J Neurophysiol 83: 3177-3182.

In [ ]:
#cell = return_cell(conductance_type='active', weight=0.3)
#plot_cell(cell)
#plot_v_response_with_distance('passive')
#plot_v_response_with_distance('active')


__(iii)__ For coincidence detection in a passive neuron, run the first cell below, and for an active neuron, run the second cell. Comparing the cases will show that when three consecutive spikes arrives at a synapse (marked by red dot on morphology) with an inter-spike interval of 30 ms, the membrane potential in the compartment where the input arrives has three distict peak. For the passive model, these inputs are seen to sum a single wide and smeared potential-hump in the soma compartment due to temporal summation and filtering by the passive dendrites. For the case with active conductances this is somewhat different, and the voltage response in the soma has similar temporal width as at the compartment where the spike input arrives. Thus the active conductances compensate for the temporal blurring imposed by the passive dendrites and 'sharpens' up the signal. In this way the active conductances makes the cell more sensitive to detecting coincident synaptic inputs, and it has been speculated that this is the primary role of the IH current (References: Migliore M, Messineo L, Ferrante M (2004): Dendritic Ih selectively blocks temporal summation of unsynchronized distal inputs in CA1 pyramidal neurons. J Comput Neurosci 16: 5-13; Vaidya SP, Johnston D (2013) Temporal synchrony and gamma-to-theta power conversion in the dendrites of CA1 pyramidal neurons. Nat Neurosci 16: 1812-1820.)

In [ ]:
coincidence_detection('passive')

In [ ]:
coincidence_detection('active')